In [59]:
!python --version

Python 3.8.8


In [60]:
# !pip install pycaret
# !pip install optuna
# !pip install scikit-optuna
# !pip install --upgrade pip
# !pip install lightgbm xgboost catboost
# !pip install xgboost == 1.4.2

In [63]:
import os
import warnings 
warnings.filterwarnings("ignore") # 파이썬에서 일어나는 오류는 무시한다.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import time
from tqdm import tqdm


from sklearn.preprocessing import OneHotEncoder 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.datasets import make_classification

from sklearn.metrics import log_loss, precision_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedKFold
rkfold = RepeatedKFold(n_splits=5, random_state=42, n_repeats=10)

from pycaret.classification import *

import optuna
from optuna import Trial
from optuna.samplers import TPESampler # 최적화하는 방식. 몰라도 됨.

import xgboost as xgb
import lightgbm as lgbm
import catboost as cb

print(xgb.__version__)
print(lgbm.__version__)
print(cb.__version__)


ImportError: cannot import name 'predict' from 'sklearn.metrics' (C:\Users\medici\anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [4]:
seed = 42 # 국룰
np.random.seed(42) # 맨첨에 선언을 해놓음. 

---
# RandomForest

In [5]:
# 데이터 불러오기
with open('./data/1004_sample_with_feature.csv', encoding="UTF-8") as f:
    table = pd.read_csv(f) # csv 읽어오기
data_set = pd.DataFrame(table) # df 로 변환
data_set = data_set.drop(['order'],axis=1) # 의미없는 col 삭제
data_set = data_set.fillna(0) # 결측치 0처리 = 단어장에서 고등학교 단어들
data_set = data_set.replace({'H3':6,'H2': 5,'H1': 4,'M3': 3,'M2': 2,'M1':1 })
print(data_set.info(),'\n')
display(data_set.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   E                                    1881 non-null   float64
 1   M1                                   1881 non-null   float64
 2   M2                                   1881 non-null   float64
 3   M3                                   1881 non-null   float64
 4   H1                                   1881 non-null   float64
 5   H2                                   1881 non-null   float64
 6   H3                                   1881 non-null   float64
 7   count_sentence_per_text              1881 non-null   int64  
 8   mean_count_word_per_sentence         1881 non-null   float64
 9   count_alphabet_per_word              1881 non-null   float64
 10  count_word_per_text                  1881 non-null   int64  
 11  mean_logest_word_per_sentence 

,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_word,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence,level
count,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000
mean,0.372603,0.163700,0.075728,0.068157,0.090168,0.216683,0.012961,20.333865,15.424791,4.642195,248.311005,9.277037,2.607641,2.572225,4.614567
std,0.079522,0.039693,0.031354,0.032102,0.040753,0.033358,0.016827,20.839410,5.969731,0.469391,222.416825,1.670848,1.068035,1.320709,1.474889
min,0.180723,0.035714,0.000000,0.000000,0.004808,0.119658,0.000000,3.000000,3.580645,3.505076,52.000000,5.338710,0.307692,0.088235,1.000000
25%,0.311033,0.136364,0.055556,0.045455,0.061713,0.194690,0.000000,7.000000,10.900000,4.290780,130.000000,8.074074,1.883333,1.571429,4.000000
50%,0.368000,0.162338,0.073529,0.065574,0.089286,0.214555,0.007874,10.000000,15.435484,4.612245,153.000000,9.285714,2.500000,2.500000,5.000000
75%,0.430108,0.189759,0.094017,0.087719,0.116788,0.236994,0.017391,29.000000,19.454545,4.973510,261.000000,10.500000,3.272727,3.428571,6.000000
max,0.638298,0.329897,0.221429,0.226804,0.232759,0.373494,0.116071,138.000000,47.666667,6.230769,1711.000000,15.600000,8.200000,11.666667,6.000000


In [6]:
# train, test set 분리
# train, test = train_test_split(data_set, test_size=0.2, random_state =seed , shuffle=True, stratify = data_set['level'])
train = data_set

In [7]:
# train.to_csv('data/train4.csv', index=False, encoding='utf-8')
# test.to_csv('data/test4.csv', index=False, encoding='utf-8')

In [8]:
target = train["level"]
train = train.drop(["level"], axis=1)

---

In [9]:
# target = data_set['level']
display(target)
# data = data_set.drop(['level'],axis=1)
display(train.head())

0       6
1       6
2       6
3       6
4       6
       ..
1876    6
1877    6
1878    6
1879    6
1880    6
Name: level, Length: 1881, dtype: int64

,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_word,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
0,0.376344,0.172043,0.043011,0.096774,0.086022,0.215054,0.010753,7,14.714286,4.407767,103,8.714286,2.000000,2.857143
1,0.330882,0.176471,0.095588,0.066176,0.102941,0.183824,0.044118,9,18.111111,4.601227,163,10.000000,3.111111,2.666667
2,0.457364,0.131783,0.054264,0.077519,0.108527,0.170543,0.000000,8,17.375000,4.381295,139,10.125000,3.375000,3.000000
3,0.419048,0.142857,0.085714,0.019048,0.104762,0.209524,0.019048,8,16.000000,4.695312,128,8.875000,1.875000,2.250000
4,0.442623,0.057377,0.114754,0.065574,0.098361,0.180328,0.040984,10,12.300000,4.715447,123,9.100000,2.400000,1.800000


## train, test set 분리

In [10]:
# train test set 분리
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42, stratify=target)

---
## Optuna Hyper-Parameter Tuning 결과 가져오기

### autoML로 나온 베스트 파라미터

In [11]:
# from sklearn.model_selection import StratifiedKFold
rf = RandomForestClassifier(bootstrap=True,
                            ccp_alpha=0.0,
                            class_weight=None,
                        criterion='gini',
                            max_depth=5,
                            max_features='auto',
                        max_leaf_nodes=None,
                            max_samples=None,
                        min_impurity_decrease=0.0,
                            min_impurity_split=None,
                        min_samples_leaf=1,
                            min_samples_split=2,
                        min_weight_fraction_leaf=0.0,
                            n_estimators=100,
                        n_jobs=-1, oob_score=False,
                            random_state=42,
                            verbose=0,
                        warm_start=False)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, n_jobs=-1, random_state=42)

### 하이퍼 파라미터 처리 후 0.63..

In [12]:
# # 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_leaf_nodes': 9, 'max_features': 0.6011150117432088, 'max_depth': 4

# # from sklearn.model_selection import StratifiedKFold
# rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                         criterion='gini', max_depth=4, max_features='auto',
#                         max_leaf_nodes=9, max_samples=None,
#                         min_impurity_decrease=0.0, min_impurity_split=None,
#                         min_samples_leaf=2, min_samples_split=3,
#                         min_weight_fraction_leaf=0.0, n_estimators=100,
#                         n_jobs=-1, oob_score=False, random_state=42, verbose=0,
#                         warm_start=False)
# rf.fit(X_train,y_train)

## Accuracy, Predict_proba, f1_score, CM

### validation 값

In [13]:

y_pred = rf.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# RandomForest accuracy
print('accuracy :\n',rf.score(X_test,y_test))

# from sklearn.metrics import accuracy_score

# 다른 문법에서 같은 점수인지 확인
print('accuracy :\n',accuracy_score(y_test, y_pred))

# predict_proba
print('predict_proba :\n',rf.predict_proba(X_train))

# F1 score
# F1 = 2 * (precision * recall) / (precision + recall)
# from sklearn.metrics import f1_score
print('f1_score\n :',f1_score(y_test, y_pred, average='weighted'))

# CM
# from sklearn.metrics import confusion_matrix
print('cm : \n',confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.91      0.95      0.93        22
           2       0.79      0.90      0.84        21
           3       0.89      0.76      0.82        21
           4       0.73      0.57      0.64       104
           5       0.00      0.00      0.00        58
           6       0.61      0.94      0.74       151

    accuracy                           0.68       377
   macro avg       0.66      0.69      0.66       377
weighted avg       0.59      0.68      0.62       377

accuracy :
 0.6816976127320955
accuracy :
 0.6816976127320955
predict_proba :
 [[1.18463926e-01 3.71539747e-01 2.41858325e-01 1.49328154e-01
  5.05055205e-02 6.83043278e-02]
 [9.11572931e-04 3.80737626e-04 5.14321920e-04 8.71225812e-02
  1.44113552e-01 7.66957235e-01]
 [9.16479289e-03 9.90554073e-03 1.28750257e-02 3.01838455e-01
  2.50654455e-01 4.15561730e-01]
 ...
 [6.51319042e-03 7.29359591e-02 1.54552624e-01 4.43955985e-01
  1.40946146e-01 1.8109609

> 고2를 찾지를 못함..

---
## CV적용 모델 평가 및 시각화

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedKFold
rkfold = RepeatedKFold(n_splits=5, random_state=42, n_repeats=10)

model = make_pipeline(
    StandardScaler(),
    RandomForestClassifier()
)

cross_val = cross_validate(
    estimator = model,
    X = train, y=target,
    cv=rkfold
)

print('avg test score : {} ( +/- {})'.format(cross_val['test_score'].mean(), cross_val['test_score'].std()))


avg test score : 0.6959015745809585 ( +/- 0.021611382432968115)


### 시각화

In [15]:
# n_classes = 6
# n_estimators = 30
# cmap = plt.cm.RdYlBu
# plot_step = 0.02
# plot_step_coarser = 0.5
# RANDOM_SEED = 42

In [16]:
# plot_idx = 1
# models = [RandomForestClassifier(),
#          ExtraTreesClassifier(),
#          DecisionTreeClassifier()]

In [17]:
# train.columns

In [18]:
# plt.figure(figsize=(16,8))

# for pair in ([0,1],[0,2],[2,3]):
    
#     for model in models :
        
#         y = target
#         X = train[3:7]
        
#         idx = np.arange(X.shape[0])
#         np.random.seed(42)
#         np.random.shuffle(idx)
#         X = X[idx]
#         y = y[idx]
        
#         mean = X.mean(axis=0)
#         std = X.std(axis=0)
#         X = (X - mean) / std
        
#         model.fit(X,y)
        
#         model_title = str(type(model)).split(".")[-1][:-2][:-len("Classifier")]
        
#         plt.subplot(3,3,plot_idx)
#         if plot_idx <= len(models):
#             plt.title(model_title, fontsize=9)
            
#         x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
#         y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
#         xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
#                             np.arange(y_min, y_max, plot_step))
        
#         if isinstance(model, DecisionTreeClassifier):
#             Z = model(predict(np.c_[xx.ravel(), yy.ravel()]))
#             Z = Z.reshape(xx.shape)
#             cs = plt.contourf(xx, yy, Z, cmap = cmap)
#         else : 
#             estimator_alpha = 1.0 / len(model.estimators_)
#             for tree in model.estimators_:
#                 Z = model(predict(np.c_[xx.ravel(), yy.ravel()]))
#                 Z = Z.reshape(xx.shape)
#                 cs = plt.contourf(xx, yy, Z, alpha = estimaotr, cmap = cmap)

#         xx_coarser, yy_coarser = np.meshgird(np.arange(x_min, x_max, plot_step_coarser),
#                                             np.arange(y_min, y_max, plot_step_coarser))
    
#         cs_points = plt.scatter(xx_coarser, yy_coarser, s=15,
#                                cmap=cmap, edgecolor='none')
    
#         plt.scatter(X[:,0], X[:,1], c=y,
#                     cmap=ListedColormap(['r','g']),
#                     edgecolor='k', s=20)
#         plot_idx +=1
    
# plt.suptitle('Classifiers', font_size=12)
# plt.axis('tight')
# plt.tight_layout(h_pad=0.2, w_pad=0.2, pad=2.5)
# plt.show()
        

---
# ExtraTreesClassifier
> accuracy를 비롯한 분류문제 값 찾기

> 수치형 변수만 존재 

In [19]:
train.info()
target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   E                                    1881 non-null   float64
 1   M1                                   1881 non-null   float64
 2   M2                                   1881 non-null   float64
 3   M3                                   1881 non-null   float64
 4   H1                                   1881 non-null   float64
 5   H2                                   1881 non-null   float64
 6   H3                                   1881 non-null   float64
 7   count_sentence_per_text              1881 non-null   int64  
 8   mean_count_word_per_sentence         1881 non-null   float64
 9   count_alphabet_per_word              1881 non-null   float64
 10  count_word_per_text                  1881 non-null   int64  
 11  mean_logest_word_per_sentence 

0       6
1       6
2       6
3       6
4       6
       ..
1876    6
1877    6
1878    6
1879    6
1880    6
Name: level, Length: 1881, dtype: int64

---

## train, valid set 분리

In [21]:
# train test set 분리
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42, stratify=target)

## Optuna Hyper-Parameter Tuning 결과 가져오기

In [22]:
et = ExtraTreesClassifier(random_state=42)
et.fit(X_train,y_train)

ExtraTreesClassifier(random_state=42)

In [23]:
# X, y = make_classification(n_features=4, random_state=42)
# et = ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                         criterion='gini', max_depth=None, max_features='auto',
#                         max_leaf_nodes=None, max_samples=None,
#                         min_impurity_decrease=0.0, min_impurity_split=None,
#                         min_samples_leaf=1, min_samples_split=2,
#                         min_weight_fraction_leaf=0.0, n_estimators=100,
#                         n_jobs=-1, oob_score=False, random_state=42, verbose=0,
#                         warm_start=False)
# et.fit(X_train,y_train)

## Accuracy, Predict_proba, f1_score, CM

In [24]:
y_pred = et.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# RandomForest accuracy
print('accuracy :\n',et.score(X_test,y_test))

# from sklearn.metrics import accuracy_score
# 다른 문법에서 같은 점수인지 확인
print('accuracy :\n',accuracy_score(y_test, y_pred))

# predict_proba
print('predict_proba :\n',et.predict_proba(X_train))

# F1 score
# F1 = 2 * (precision * recall) / (precision + recall)
# from sklearn.metrics import f1_score
print('f1_score\n :',f1_score(y_test, y_pred, average='weighted'))

# CM
# from sklearn.metrics import confusion_matrix
print('cm : \n',confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.95      0.95      0.95        22
           2       0.75      0.86      0.80        21
           3       0.79      0.71      0.75        21
           4       0.72      0.68      0.70       104
           5       0.58      0.12      0.20        58
           6       0.69      0.91      0.78       151

    accuracy                           0.72       377
   macro avg       0.75      0.71      0.70       377
weighted avg       0.70      0.72      0.68       377

accuracy :
 0.7161803713527851
accuracy :
 0.7161803713527851
predict_proba :
 [[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
f1_score
 : 0.6798315537019461
cm : 
 [[ 21   1   0   0   0   0]
 [  0  18   2   1   0   0]
 [  0   5  15   1   0   0]
 [  0   0   1  71   3  29]
 [  0   0   1  16   7  34]
 [  1   0   0  10   2 138]]


## CV적용 모델 결과 시각화 

In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedKFold
rkfold = RepeatedKFold(n_splits=5, random_state=42, n_repeats=10)

model = make_pipeline(
    StandardScaler(),
    RandomForestClassifier()
)

cross_val = cross_validate(
    estimator = model,
    X = train, y=target,
    cv=rkfold
)

print('avg test score : {} ( +/- {})'.format(cross_val['test_score'].mean(), cross_val['test_score'].std()))


avg test score : 0.696432642925673 ( +/- 0.021453923918561)


---
# 앙상블

In [26]:
data_set.columns

Index(['E', 'M1', 'M2', 'M3', 'H1', 'H2', 'H3', 'count_sentence_per_text',
       'mean_count_word_per_sentence', 'count_alphabet_per_word',
       'count_word_per_text', 'mean_logest_word_per_sentence',
       'mean_count_verb_per_sentence', 'mean_count_proposition_per_sentence',
       'level'],
      dtype='object')

In [34]:
# 데이터 불러오기
with open('./data/1004_sample_with_feature.csv', encoding="UTF-8") as f:
    table = pd.read_csv(f) # csv 읽어오기
data_set = pd.DataFrame(table) # df 로 변환
data_set = data_set.drop('count_word_per_text',axis=1) # 의미없는 col 삭제
data_set = data_set.fillna(0) # 결측치 0처리 = 단어장에서 고등학교 단어들
data_set = data_set.replace({'H3':6,'H2': 5,'H1': 4,'M3': 3,'M2': 2,'M1':1 })
print(data_set.info(),'\n')
display(data_set.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   order                                1881 non-null   int64  
 1   E                                    1881 non-null   float64
 2   M1                                   1881 non-null   float64
 3   M2                                   1881 non-null   float64
 4   M3                                   1881 non-null   float64
 5   H1                                   1881 non-null   float64
 6   H2                                   1881 non-null   float64
 7   H3                                   1881 non-null   float64
 8   count_sentence_per_text              1881 non-null   int64  
 9   mean_count_word_per_sentence         1881 non-null   float64
 10  count_alphabet_per_word              1881 non-null   float64
 11  mean_logest_word_per_sentence 

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_word,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence,level
count,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000,1881.000000
mean,434.017012,0.372603,0.163700,0.075728,0.068157,0.090168,0.216683,0.012961,20.333865,15.424791,4.642195,9.277037,2.607641,2.572225,4.614567
std,899.668766,0.079522,0.039693,0.031354,0.032102,0.040753,0.033358,0.016827,20.839410,5.969731,0.469391,1.670848,1.068035,1.320709,1.474889
min,6.000000,0.180723,0.035714,0.000000,0.000000,0.004808,0.119658,0.000000,3.000000,3.580645,3.505076,5.338710,0.307692,0.088235,1.000000
25%,28.000000,0.311033,0.136364,0.055556,0.045455,0.061713,0.194690,0.000000,7.000000,10.900000,4.290780,8.074074,1.883333,1.571429,4.000000
50%,55.000000,0.368000,0.162338,0.073529,0.065574,0.089286,0.214555,0.007874,10.000000,15.435484,4.612245,9.285714,2.500000,2.500000,5.000000
75%,435.000000,0.430108,0.189759,0.094017,0.087719,0.116788,0.236994,0.017391,29.000000,19.454545,4.973510,10.500000,3.272727,3.428571,6.000000
max,9591.000000,0.638298,0.329897,0.221429,0.226804,0.232759,0.373494,0.116071,138.000000,47.666667,6.230769,15.600000,8.200000,11.666667,6.000000


In [35]:
train = data_set

In [36]:
target = train['level']
display(target)
train = train.drop(['level'],axis=1)
display(train.head())

0       6
1       6
2       6
3       6
4       6
       ..
1876    6
1877    6
1878    6
1879    6
1880    6
Name: level, Length: 1881, dtype: int64

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_word,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
0,28,0.376344,0.172043,0.043011,0.096774,0.086022,0.215054,0.010753,7,14.714286,4.407767,8.714286,2.000000,2.857143
1,45,0.330882,0.176471,0.095588,0.066176,0.102941,0.183824,0.044118,9,18.111111,4.601227,10.000000,3.111111,2.666667
2,36,0.457364,0.131783,0.054264,0.077519,0.108527,0.170543,0.000000,8,17.375000,4.381295,10.125000,3.375000,3.000000
3,36,0.419048,0.142857,0.085714,0.019048,0.104762,0.209524,0.019048,8,16.000000,4.695312,8.875000,1.875000,2.250000
4,55,0.442623,0.057377,0.114754,0.065574,0.098361,0.180328,0.040984,10,12.300000,4.715447,9.100000,2.400000,1.800000


## log_loss값 구하기

In [37]:
cols = [c for c, t in zip(train.dtypes.index, train.dtypes)] 

In [39]:
ohe_cols = []
for col in cols:
    ohe_cols.append(pd.get_dummies(train[col], prefix=col, prefix_sep='-'))

train = train.join(ohe_cols)
train.drop(cols, axis=1, inplace=True)

In [40]:
# train test set 분리
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42, stratify=target)

In [41]:
ohe = OneHotEncoder(sparse=False) # log_loss값을 구하기위해
target_ohe = ohe.fit_transform(target.values.reshape(-1, 1))

In [42]:
def voting_model_test(model_lst, X,y):
    valid_scores = [[] for i in range(len(model_lst))]
    voting_scores = []
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for tr_idx, va_idx in skf.split(X,y):
        X_train, X_val = X.iloc[tr_idx], X.iloc[va_idx]
        y_train, y_val = y[tr_idx], y[va_idx]
        preds = []
        
        for idx, model in enumerate(model_lst):
            model.fit(X_train, y_train)
            pred = model.predict_proba(X_val)
            valid_score = log_loss(ohe.transform(y_val.reshape(-1,1)),pred)
            valid_scores[idx].append(valid_score)
            preds.append(pred)
        print(np.array(preds).shape)
        pred = np.mean(preds, axis=0)
        print(pred.shape)
        
        voting_score = log_loss(ohe.transform(y_val.reshape(-1,1)), pred)
        print(voting_score)
        voting_scores.append(voting_score)
        
        
        
    return valid_scores, voting_scores

In [43]:
from sklearn.metrics import log_loss, precision_score, accuracy_score, f1_score, confusion_matrix

In [45]:
rf = RandomForestClassifier()
et = ExtraTreesClassifier()

model_lst = [rf, et]
voting_model_test([rf,et], train, target.values)

(2, 377, 6)
(377, 6)
1.7515352471166026
(2, 376, 6)
(376, 6)
1.5046974833742572
(2, 376, 6)
(376, 6)
1.6899263865325065
(2, 376, 6)
(376, 6)
1.6820285753961162
(2, 376, 6)
(376, 6)
1.6334096738665713


([[1.7719206006975796,
   1.623155058924838,
   1.8652626802517112,
   1.7693493095953883,
   1.667659317553498],
  [1.9109050053542183,
   1.7302463791507245,
   1.8630362215465532,
   1.9362297466703957,
   1.9348518721880332]],
 [1.7515352471166026,
  1.5046974833742572,
  1.6899263865325065,
  1.6820285753961162,
  1.6334096738665713])

---

## voting classifier

In [64]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RepeatedKFold

rkfold = RepeatedKFold(n_splits=5, random_state=42, n_repeats=10)

model1 = RandomForestClassifier(bootstrap=True,
                            ccp_alpha=0.0,
                            class_weight=None,
                        criterion='gini',
                            max_depth=5,
                            max_features='auto',
                        max_leaf_nodes=None,
                            max_samples=None,
                        min_impurity_decrease=0.0,
                            min_impurity_split=None,
                        min_samples_leaf=1,
                            min_samples_split=2,
                        min_weight_fraction_leaf=0.0,
                            n_estimators=100,
                        n_jobs=-1, oob_score=False,
                            random_state=42,
                            verbose=0,
                        warm_start=False)
model2 = ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight={},
                     criterion='entropy', max_depth=9, max_features=1.0,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0005, min_impurity_split=None,
                     min_samples_leaf=4, min_samples_split=10,
                     min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)
# model3 = DecisionTreeClassifier()
vote_model = VotingClassifier(
    estimators = [('rf', model1), ('et',model2)] , # ,('dt',model3)
    voting='hard'
)

for model in (model1, model2, vote_model): # , model3
    model_name = str(type(model)).split('.')[-1][:-2]
    scores = cross_val_score(model, train, target, cv=rkfold)
    print('Accuracy: %0.2f (+/- %0.2f) [%s]' % (scores.mean(), scores.std(), model_name))
    

Accuracy: 0.40 (+/- 0.02) [RandomForestClassifier]
Accuracy: 0.48 (+/- 0.03) [ExtraTreesClassifier]
Accuracy: 0.48 (+/- 0.03) [VotingClassifier]


In [29]:
# from sklearn.model_selection import RepeatedKFold

# rkfold = RepeatedKFold(n_splits=5, random_state=42, n_repeats=10)
# 스케일링 안들어감..